# Car Price prediction

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*В качестве шаблона использован нутбук Baseline*   


> **Baseline** создается больше как шаблон, где можно посмотреть, как происходит обращение с входящими данными и что нужно получить на выходе. При этом ML начинка может быть достаточно простой. Это помогает быстрее приступить к самому ML, а не тратить ценное время на инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если наше решение хуже baseline -  мы явно делаем что-то не так и стоит попробовать другой путь) 

## В baseline мы сделаем следующее:
* Построим "наивную"/baseline модель, предсказывающую цену по модели и году выпуска (с ней будем сравнивать другие модели)
* Обработаем и отнормируем признаки
* Сделаем первую модель на основе градиентного бустинга с помощью CatBoost
* Сделаем вторую модель на основе нейронных сетей и сравним результаты
* Сделаем multi-input нейронную сеть для анализа табличных данных и текста одновременно
* Добавим в multi-input сеть обработку изображений
* Осуществим ансамблирование градиентного бустинга и нейронной сети (усреднение их предсказаний)

In [ ]:
# !pip install -q tensorflow==2.3

In [ ]:
#аугментации изображений
# !pip install albumentations -q

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import string
from collections import Counter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords, words
from nltk.stem.snowball import RussianStemmer
from nltk.util import ngrams
from tqdm import tqdm


# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
# зафиксирован RANDOM_SEED, чтобы эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
train.nunique()

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [ ]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

# EDA

Проведем анализ данных для того, чтобы понимать, сможет ли с этими данными работать наш алгоритм.
Посмотрим, как выглядят распределения числовых признаков:

In [ ]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(3, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize = (columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins = 20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Итого:
* CatBoost сможет работать с признаками и в таком виде, но для нейросети нужны нормированные данные.

## PreProc Tabular Data

In [ ]:
#используем текстовые признаки как категориальные 
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', 'name',
                        'engineDisplacement', 'enginePower', 'numberOfDoors', 
                        'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 
                        'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate']

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

Посмотрим есть ли зависимость между сроком владения и датой производства авто

In [ ]:
data[['Владение', 'productionDate', 'Владельцы' ]].sample(10)

Видно что Владение зависит от количества владельцев. Можно заполнить пустые значения по дате производства и количеству владельцев

In [ ]:
'''Функция для перевода чисел из строки в целое значение'''
def num_extract(text):
    pattern = re.compile(r"\d+") # паттерн для чисел
    num_list = pattern.findall(str(text)) # находим все числа
    if len(num_list)>1: # если чисел больше 1 то первое значение это год, второе-месяц
        return int(num_list[0])*12+int(num_list[1]) # вычисляем сколько всего месяцев
    else:
        return int(num_list[0])

In [ ]:
 # проверим пустые поля в категориальных признаках
for column in categorical_features:
    print(column, ' - ', data[column].isnull().sum())

In [ ]:
'''Извлечение числовых значений из текста''' 
# В полях 'engineDisplacement' и 'enginePower' оставляем только числа 
for feature in ['engineDisplacement', 'enginePower']:
    data[feature]=data[feature].apply(lambda x: x[:3])
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: 2.0 if x == 'und' else x)
# Поработаем с признаком Владельцы: заполняем пустое поле и переводим значения в числа 
data['Владельцы'].fillna(3, inplace=True)
data['Владельцы'] = data['Владельцы'].apply(lambda x: 1 if x == '1\xa0владелец' 
                                            else 2 if x == '2\xa0владельца' else 3)

# переводим признак 'Владение' в числовое значение количества месяцев
# сначало заполним пустые значение возрастом авто на 2020 год / количество владельцев
data['Владение'].fillna((2020 - data['productionDate'])*12/data['Владельцы'], inplace=True)
# переводим строки в числовые значения в месяцах
data['Владение'] = data['Владение'].apply(lambda x: num_extract(x))

# Переводим предыдущие признаки в числовую категорию
for feature in ['engineDisplacement', 'enginePower']:
    data[feature]=data[feature].astype('float').round(1)
    
for feature in ['engineDisplacement', 'enginePower','Владение','Владельцы']:
    numerical_features.append(feature)
        
# Удаляем эти два поля из категориальных признаков
for feature in ['engineDisplacement', 'enginePower','Владение','Владельцы']:
    categorical_features.remove(feature)

Поработаем с признаком vehicleConfiguration 

In [ ]:
english_stopwords = stopwords.words("english") + ['лс','квт']
spec_chars = string.punctuation + '«»—…’‘”“©' #перечень заков пунктуации

# Зададим функцию для удаления спец символов
def remove_chars_from_text(text, chars):
    return "".join([ch for ch in text if ch not in chars])

In [ ]:
# Зададим функцию для токенизации
def text_tokenizer(text):
    Text_ = text.strip()

    Text_ = Text_.lower()

    # Удалим все спец символы
    Text_ = remove_chars_from_text(Text_, spec_chars)

    # Удалим все цифры
    Text_ = remove_chars_from_text(Text_, string.digits)

    Text_ = Text_.replace('\n',' ').replace('\t',' ')

    # Токенизируем текст
    tokens = word_tokenize(Text_)

    # Список токенов преобразовываем к классу Text
    token_text = nltk.Text(tokens)

    # Удаляем стоп-слова
    filtered_token_text = [w for w in token_text if not w in english_stopwords]
    
    return filtered_token_text

In [ ]:
data['vehicleConfiguration'].apply(lambda x: text_tokenizer(x))

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id'], axis = 1, inplace=True)
      
           
    # убираем из признака 'name' данные, которые уже есть 
    # в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission')    
    pattern1 = r"\d.\d \w+ \([^()]*\)" 
    pattern2 = r"\d.\d\w \w+ \([^()]*\)"   
    df_output['name'] = df_output['name'].apply(lambda x: re.sub(pattern1, "", x))  
    df_output['name'] = df_output['name'].apply(lambda x: re.sub(pattern2, "", x))   
        
        
    # ################### Numerical Features ################################
    # Далее заполняем пропуски в числовых полях
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
       
       
    # ################### Feature Engineering ####################################################
                           
    df_output['mileage'] = df_output['mileage']**(0.5) # Лучшая корреляция с price если брать mileage в сетпени 0.5  
    df_output['modelDateNorm'] = np.log(2020 - df_output['modelDate']) #нормализуем дату производства авто
    
    # наличие xDrive
    df_output['xDrive'] = df_output['name'].apply(lambda x: 1 if re.search(r'xDrive', x) else 0)
    df_output['name'] = df_output['name'].apply(lambda x: re.sub(r'xDrive', "", x))
    
    
    # обрабатываем признак vehicleConfiguration
    df_output['vehicleConfiguration'] = df_output['vehicleConfiguration'].apply(lambda x: text_tokenizer(x))
    
    Veh_Con_dum = df_output['vehicleConfiguration'].apply(lambda x: str(x).replace('[','').
                                                       replace(']','')).str.get_dummies(sep=', ').reset_index(drop=True)
    
    # объединим новые дамми признаки с основным датасетом
    df_output = pd.concat([df_output, Veh_Con_dum], axis=1)
    
    
    # ################### Categorical Features ######################## 
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    
    
    # ################### Clean #######################################
    # убираем признаки которые не нужны более 
    df_output.drop(['vehicleConfiguration'], axis = 1, inplace=True)   
    
    
    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

## Split data

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [ ]:
test_data.info()

# Model 2: CatBoostRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.15, 
                                                    shuffle=True, 
                                                    random_state=RANDOM_SEED)

Целевую переменную прологарифмируем, чтобы распрделение было более нормальным, это влияет на качество предсказаний регрессии. 

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          depth=10,
#                           learning_rate = 0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

In [ ]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

## Submission

In [ ]:
sub_predict_catboost = np.exp(model.predict(X_sub))
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission.csv', index=False)

## Выводы:
* Очень хорошее улучшение по сравнению с наивной моделью
* Можно было бы еще поработать над обработкой признаков, в частности оценить выбросы, выделить еще больше новых признаков.

# Model 3: Tabular NN

Построим обычную сеть:

In [ ]:
X_train.head(5)

## Simple Dense NN

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [ ]:
model.summary()

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5',
                             monitor=['val_MAPE'], verbose=0,
                             mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50,
                          restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### Fit

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

### Выводы:
* Нейросеть отработала лучше наивной модели, но хуже CatboostRegressor
* Можно было бы еще поработать над обработкой признаков. В частности оценить выбросы, выделить еще больше новых признаков.
* Проведена следующая работа для улучшения модели:
##### CatBoostRegressor:
* в модель подавался логарифм от целевой переменной, а затем на предикте бралась экспанента.
##### Tabular:
* В нейросеть подаются данные с распределением, близким к нормальному. Создан признак modelDateNorm = np.log(2020 - data['modelDate']). Обработан признак 'mileage' - возведен в степень 0.5. 
* Извлечение числовых значений из текста: Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.
* Cокращение размерности признака 'name'. Удалены данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission'). Выделено наличие xDrive в качестве отдельного признака.
* Проведена обработка признака 'vehicleConfiguration' с токенизацией и выделением основных значений.

# Model 4: NLP + Multiple Inputs

In [ ]:
data.description[2]

In [ ]:
# смотрим на самые частые слова в описании
text = ' '.join(data['description'].values)
text_trigrams = [i for i in ngrams(text.split(' '), 5)]
text_trigrams
my_list = Counter(text_trigrams).most_common(50)
my_list[:20]

In [ ]:
# поработаем с самыми часто встречаемыми выражениями
data['description'] = data['description'].str.replace('автомобилей с пробегом','автоспробег')
data['description'] = data['description'].str.replace('ждут Вас на сайте rolf-probeg','сайт')
data['description'] = data['description'].str.replace('в мобильном приложении','приложение')
data['description'] = data['description'].str.replace('проверенных',' ')
data['description'] = data['description'].str.replace('с гарантией','гарант')


In [ ]:
# TOKENIZER
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

## Lemming, Tokinaezer, stop words

In [ ]:
stop_words = set(stopwords.words("russian"))
lemmatizer = WordNetLemmatizer()
stemmer = RussianStemmer(False)

In [ ]:
def clean_text(x):
    '''Очистка текста от знаков пунктуации, скобок, чисел...'''
    x = x.lower()
    # Удалим все спец символы
    x = remove_chars_from_text(x, spec_chars)

    # Удалим все цифры
    x = remove_chars_from_text(x, string.digits)
    
    # Удаляем остальные символы
    x = x.replace('\n',' ').replace('\t',' ')
    x = re.sub('\[.*?\]', '', x) # brackets
    
    #токенизация
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(x)  
    
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stop_words]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(w for w in lemma_words)

In [ ]:
data['description'] = data['description'].apply(lambda x: clean_text(x))
data['description'][2]

### Tokenizer

Создаём словарь 

In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [ ]:
tokenize.word_index

In [ ]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)
print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [ ]:
# вот так теперь выглядит наш текст
print(text_train.iloc[6])
print(text_train_sequences[6])

## RNN NLP

строим сеть для обработки текста. Для простоты в примере используем LSTM:

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

## MLP

еть построена «без головы». Добавим ещё одну сеть — созданную нами ранее для табличных данных:

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

## Multiple Inputs NN

Объединяем сети в Multi-Input сеть, то есть сеть, которая позволяет брать на вход несколько сетей и объединять их результаты. За объединение отвечает слой L.concatenate:

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [ ]:
model.summary()

### Fit

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', 
                             monitor=['val_MAPE'], 
                             verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10,
                          restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:,0]
sample_submission.to_csv('nn2_submission.csv', index=False)

## Выводы:
* NLP показала также результат лучше наивной модели, но хуже CatboostRegressor
* Можно было бы еще попробовать другие методы обучения (Fine-tuning, transfer-learning)
* Проведена следующая работа для улучшения модели:
    1. Из описания выделены наиболее часто встречающиеся выражения и заменены на кодовые слова
    2. Проведена редобработка текста с лемматизацией и стеммнигом.
    3. Предаврительно создан алгоритм очистки и аугментации текста.

# Model 5: Добавляем картинки

## Data

In [ ]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

## albumentations

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,  
    Rotate, CenterCrop, GaussianBlur,RGBShift, FancyPCA, Resize,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

Закомментированный ниже код дал результаты лучше для модели 5, но после сабмита паблик скор был ниже на 0.003

In [ ]:
# augmentation = Compose([
#     HorizontalFlip(p=0.5), # p=0.5
#     Rotate(limit=30, interpolation=1, border_mode=4, value=None, 
#            mask_value=None, always_apply=False, p=0.5),
#     OneOf([
#         CenterCrop(height=240, width=320),
#         CenterCrop(height=220, width=300),
#     ],p=0.5),
#     OneOf([
#         CLAHE(clip_limit=2),
#         IAASharpen(),
#         IAAEmboss(),
#         RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
#         RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1) 
#     ], p=0.5),#p=0.3
#     GaussianBlur(p=0.05),
#     HueSaturationValue(p=0.5),#p=0.3
#     RGBShift(p=0.5),
#     FancyPCA(alpha=0.1, always_apply=False, p=0.5),
#     Resize(240, 320)
# ], p=1)

In [ ]:
#пример взят из официальной документации: https://albumentations.readthedocs.io/en/latest/examples.html
augmentation = Compose([
    HorizontalFlip(),
    OneOf([
        IAAAdditiveGaussianNoise(),
        GaussNoise(),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),
        IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    ], p=0.3),
    HueSaturationValue(p=0.3),
], p=1)

In [ ]:
#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
        augment_dict = augmentation(image = images[i])
        augmented_image = augment_dict['image']
        augmented_images[i] = augmented_image
        print('')
    return augmented_images

## tf.data.Dataset
Если все изображения мы будем хранить в памяти, то может возникнуть проблема ее нехватки. Не храните все изображения в памяти целиком!

Метод .fit() модели keras может принимать либо данные в виде массивов или тензоров, либо разного рода итераторы, из которых наиболее современным и гибким является [tf.data.Dataset](https://www.tensorflow.org/guide/data). Он представляет собой конвейер, то есть мы указываем, откуда берем данные и какую цепочку преобразований с ними выполняем. Далее мы будем работать с tf.data.Dataset.

Dataset хранит информацию о конечном или бесконечном наборе кортежей (tuple) с данными и может возвращать эти наборы по очереди. Например, данными могут быть пары (input, target) для обучения нейросети. С данными можно осуществлять преобразования, которые осуществляются по мере необходимости ([lazy evaluation](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BD%D0%B8%D0%B2%D1%8B%D0%B5_%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)).

`tf.data.Dataset.from_tensor_slices(data)` - создает датасет из данных, которые представляют собой либо массив, либо кортеж из массивов. Деление осуществляется по первому индексу каждого массива. Например, если `data = (np.zeros((128, 256, 256)), np.zeros(128))`, то датасет будет содержать 128 элементов, каждый из которых содержит один массив 256x256 и одно число.

`dataset2 = dataset1.map(func)` - применение функции к датасету; функция должна принимать столько аргументов, каков размер кортежа в датасете 1 и возвращать столько, сколько нужно иметь в датасете 2. Пусть, например, датасет содержит изображения и метки, а нам нужно создать датасет только из изображений, тогда мы напишем так: `dataset2 = dataset.map(lambda img, label: img)`.

`dataset2 = dataset1.batch(8)` - группировка по батчам; если датасет 2 должен вернуть один элемент, то он берет из датасета 1 восемь элементов, склеивает их (нулевой индекс результата - номер элемента) и возвращает.

`dataset.__iter__()` - превращение датасета в итератор, из которого можно получать элементы методом `.__next__()`. Итератор, в отличие от самого датасета, хранит позицию текущего элемента. Можно также перебирать датасет циклом for.

`dataset2 = dataset1.repeat(X)` - датасет 2 будет повторять датасет 1 X раз.

Если нам нужно взять из датасета 1000 элементов и использовать их как тестовые, а остальные как обучающие, то мы напишем так:

`test_dataset = dataset.take(1000)
train_dataset = dataset.skip(1000)`

Датасет по сути неизменен: такие операции, как map, batch, repeat, take, skip никак не затрагивают оригинальный датасет. Если датасет хранит элементы [1, 2, 3], то выполнив 3 раза подряд функцию dataset.take(1) мы получим 3 новых датасета, каждый из которых вернет число 1. Если же мы выполним функцию dataset.skip(1), мы получим датасет, возвращающий числа [2, 3], но исходный датасет все равно будет возвращать [1, 2, 3] каждый раз, когда мы его перебираем.

tf.Dataset всегда выполняется в graph-режиме (в противоположность eager-режиму), поэтому либо преобразования (`.map()`) должны содержать только tensorflow-функции, либо мы должны использовать tf.py_function в качестве обертки для функций, вызываемых в `.map()`. Подробнее можно прочитать [здесь](https://www.tensorflow.org/guide/data#applying_arbitrary_python_logic).

In [ ]:
data.description[6]

In [ ]:
# NLP part
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions),
                                  maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)


test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)


y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)


#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

## Строим сверточную сеть для анализа изображений без "головы"

In [ ]:
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
#строим нейросеть для анализа табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    ])

In [ ]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [ ]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, 
                               tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, 
                      tabular_model.input, nlp_model.input], outputs=head)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5',
                             monitor=['val_MAPE'],
                             verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE',
                          patience=10, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=20,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

## Fine tuning

In [ ]:
print("Number of layers in the base model: ", len(model.layers))

In [ ]:
# возьмем половину слоев у базовой модели
model.trainable = True

# Точная настройка, начиная с этого слоя
fine_tune_at = len(model.layers)//2

# Заморозим все слои перед тонкой надстройкой
for layer in model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
len(model.trainable_variables)

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5',
                             monitor=['val_MAPE'], 
                             verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', 
                          patience=10, 
                          restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=20,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
model.trainable = True

fine_tune_at = 100

for layer in model.layers[:fine_tune_at]:
    layer.trainable =  False

len(model.trainable_variables)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0001)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5',
                             monitor=['val_MAPE'],
                             verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', 
                          patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

In [ ]:
history = model.fit(train_dataset.batch(15),
                    epochs=25,
                    validation_data = test_dataset.batch(15),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(15))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(15))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

### Выводы:
* CV показала результат лучше наивной модели, но хуже CatboostRegressor
* Можно было бы еще попробовать другие методы обучения (transfer-learning, LR-Cycle))
* Проведена следующая работа для улучшения модели:
    1. Аугментация
    2. Использован Fine-tuning
    3. Использовалась сеть EfficientNetB4, которая показал результат хуже EfficientNetB3.

# Blend

In [ ]:
blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [ ]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

# Model Bonus: проброс признака

In [ ]:
# MLP
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

In [ ]:
# FEATURE Input
# Iput
productiondate = L.Input(shape=[1], name="productiondate")
# Embeddings layers
emb_productiondate = L.Embedding(len(X.productionDate.unique().tolist())+1, 20)(productiondate)
f_productiondate = L.Flatten()(emb_productiondate)

In [ ]:
combinedInput = L.concatenate([model_mlp.output, f_productiondate,])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_mlp.input, productiondate], outputs=head)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [ ]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5',
                             monitor=['val_MAPE'],
                             verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE',
                          patience=50,
                          restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model.fit([X_train, X_train.productionDate.values], y_train,
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([X_test, X_test.productionDate.values], y_test),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model.hdf5')
test_predict_nn_bonus = model.predict([X_test, X_test.productionDate.values])
print(f"TEST mape: {(mape(y_test, test_predict_nn_bonus[:,0]))*100:0.2f}%")

## Заключение:

### Tabular
1. В нейросеть подаются данные с распределением, близким к нормальному. Создан признак modelDateNorm = np.log(2020 - data['modelDate']).
2. Обработан признак 'mileage' - возведен в степень 0.5. 
3. Извлечение числовых значений из текста: Парсинг признаков 'engineDisplacement', 'enginePower', 'Владение' для извлечения числовых значений.
4. Cокращение размерности признака 'name'. Удалены данные, которые уже есть в других столбцах ('enginePower', 'engineDisplacement', 'vehicleTransmission').
5. Выделено наличие xDrive в качестве отдельного признака.
6. Проведена обработка признака 'vehicleConfiguration' с токенизацией и выделением основных значений.

### NLP
1. Из описания выделены наиболее часто встречающиеся выражения и заменены на кодовые слова
2. Проведена редобработка текста с лемматизацией и стеммнигом.
3. Предаврительно создан алгоритм очистки и аугментации текста.

### CV
1. Проведены различные аугментации
2. Использован Fine-tuning